In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as mc


In [2]:
pip install pandas xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.5 MB/s eta 0:00:00


In [3]:
df_new = pd.read_excel('overalldata (1).xlsx', sheet_name='Sheet1')

exclude_models = [
    "codellama-70b-instruct@anyscale",
    "codellama-7b-instruct@together-ai",
    "command-r-plus@aws-bedrock",
    "deepseek-coder-33b-instruct@together-ai",
    "phi-3-medium-4k-instruct@deepinfra"
]

filtered_df = df_new[~df_new['model'].isin(exclude_models)]

control_group = filtered_df[filtered_df['model'] == 'original_text']
other_groups = filtered_df[filtered_df['model'] != 'original_text']
combined_df = pd.concat([control_group, other_groups])

model = ols('Q("Gunning Fog") ~ C(model)', data=combined_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

# Tukey's HSD test
comp = mc.pairwise_tukeyhsd(combined_df['Gunning Fog'], combined_df['model'])
tukey_results = pd.DataFrame(data=comp.summary().data[1:], columns=comp.summary().data[0])

control_comparisons = tukey_results[tukey_results['group1'] == 'original_text']

# Sort by absolute mean difference to find the most similar models
control_comparisons_sorted = control_comparisons.sort_values(by='meandiff', key=abs)
print(control_comparisons_sorted.head())

              sum_sq     df         F        PR(>F)
C(model)  191.002691   28.0  3.775866  5.577159e-09
Residual  495.011669  274.0       NaN           NaN
            group1                         group2  meandiff   p-adj  lower  \
405  original_text  qwen-2-72b-instruct@deepinfra    2.2472  0.0034  0.363   

      upper  reject  
405  4.1315    True  


In [7]:

df_new = pd.read_excel('overalldata (1).xlsx', sheet_name='Sheet1')
exclude_models = [
    "codellama-70b-instruct@anyscale",
    "codellama-7b-instruct@together-ai",
    "command-r-plus@aws-bedrock",
    "deepseek-coder-33b-instruct@together-ai",
    "phi-3-medium-4k-instruct@deepinfra",
    "nemotron-4-340b-instruct@deepinfra",
    "codellama-13b-instruct@together-ai"
]

filtered_df = df_new[~df_new['model'].isin(exclude_models)]
summary_table = filtered_df.groupby('model')['Gunning Fog'].describe()

print(summary_table)
model = ols('Q("Gunning Fog") ~ C(model)', data=filtered_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

                                        count       mean       std    min  \
model                                                                       
claude-3-haiku@anthropic                 15.0   8.844000  1.453866   5.22   
claude-3-opus@anthropic                  15.0   8.386000  1.655263   5.73   
claude-3-sonnet@anthropic                 1.0  11.930000       NaN  11.93   
claude-3.5-sonnet@anthropic              15.0   7.834667  1.151557   5.39   
gemini-1.5-flash@vertex-ai               15.0   7.748667  1.360398   5.44   
gemini-1.5-pro@vertex-ai                 15.0   7.212667  0.998559   5.48   
gemma-2-9b-it@fireworks-ai               15.0   7.480667  1.341771   5.34   
gemma-2b-it@together-ai                  14.0   8.512143  1.530285   5.65   
gemma-7b-it@anyscale                      1.0   9.410000       NaN   9.41   
gpt-3.5-turbo@openai                     15.0   9.303333  1.234103   7.26   
gpt-4-turbo@openai                       15.0   7.874000  1.347319   5.55   

In [4]:
df_new = pd.read_excel('overalldata (1).xlsx', sheet_name='Sheet1')
exclude_models = [
    "codellama-70b-instruct@anyscale",
    "codellama-7b-instruct@together-ai",
    "command-r-plus@aws-bedrock",
    "deepseek-coder-33b-instruct@together-ai",
    "phi-3-medium-4k-instruct@deepinfra",
    "nemotron-4-340b-instruct@deepinfra",
    "codellama-13b-instruct@together-ai"
]

filtered_df = df_new[~df_new['model'].isin(exclude_models)]

summary_table = filtered_df.groupby('model')['Gunning Fog'].describe()


print(summary_table)
model = ols('Q("Gunning Fog") ~ C(model)', data=filtered_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)
anova_table.to_excel('anova_table_Gunning Fog.xlsx', index=True)

# Tukey's HSD test to compare differences between groups
tukey = mc.pairwise_tukeyhsd(filtered_df['Gunning Fog'], filtered_df['model'])

print(tukey.summary())
tukey_summary_df = pd.DataFrame(data=tukey.summary().data[1:], columns=tukey.summary().data[0])
tukey_summary_df.to_excel('tukey_summary_Gunning Fog.xlsx', index=True)

                                        count       mean       std    min  \
model                                                                       
claude-3-haiku@anthropic                 15.0   8.844000  1.453866   5.22   
claude-3-opus@anthropic                  15.0   8.386000  1.655263   5.73   
claude-3-sonnet@anthropic                 1.0  11.930000       NaN  11.93   
claude-3.5-sonnet@anthropic              15.0   7.834667  1.151557   5.39   
gemini-1.5-flash@vertex-ai               15.0   7.748667  1.360398   5.44   
gemini-1.5-pro@vertex-ai                 15.0   7.212667  0.998559   5.48   
gemma-2-9b-it@fireworks-ai               15.0   7.480667  1.341771   5.34   
gemma-2b-it@together-ai                  14.0   8.512143  1.530285   5.65   
gemma-7b-it@anyscale                      1.0   9.410000       NaN   9.41   
gpt-3.5-turbo@openai                     15.0   9.303333  1.234103   7.26   
gpt-4-turbo@openai                       15.0   7.874000  1.347319   5.55   

In [5]:
summary_table = filtered_df.groupby('model')['Gunning Fog'].describe()
anova_df = pd.DataFrame(anova_table)

output_file_path = 'summary_and_anova_output_Gunning Fog.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    summary_table.to_excel(writer, sheet_name='Summary Statistics')
    anova_df.to_excel(writer, sheet_name='ANOVA Table')

print(f"Data has been saved to {output_file_path}")

Data has been saved to summary_and_anova_output_Gunning Fog.xlsx
